In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scaling-and-smote/train_smote.pkl
/kaggle/input/scaling-and-smote/train_test_split.pkl


In [4]:
import joblib

X_train_smote, y_train_smote = joblib.load('/kaggle/input/scaling-and-smote/train_smote.pkl')
X_test = joblib.load('/kaggle/input/scaling-and-smote/train_test_split.pkl')[1]

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Reshape the data to 3D for CNN (samples, timesteps, features)
X_train_cnn = X_train_smote.values.reshape((X_train_smote.shape[0], X_train_smote.shape[1], 1))
X_test_cnn = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn, y_train_smote, epochs=10, batch_size=32)

# Extract features using CNN
cnn_features = cnn_model.predict(X_train_cnn)
cnn_features_test = cnn_model.predict(X_test_cnn)

2025-04-20 08:07:46.984874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745136467.248209      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745136467.328324      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-20 08:08:04.658639: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152

Epoch 1/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 455s 5ms/step - accuracy: 0.9302 - loss: 0.2083
Epoch 2/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 444s 5ms/step - accuracy: 0.9482 - loss: 0.1563
Epoch 3/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 468s 5ms/step - accuracy: 0.9489 - loss: 0.1495
Epoch 4/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 442s 5ms/step - accuracy: 0.9514 - loss: 0.1409
Epoch 5/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 439s 5ms/step - accuracy: 0.9515 - loss: 0.1398
Epoch 6/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 451s 5ms/step - accuracy: 0.9512 - loss: 0.1399
Epoch 7/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 444s 5ms/step - accuracy: 0.9516 - loss: 0.1384
Epoch 8/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 435s 5ms/step - accuracy: 0.9517 - loss: 0.1378
Epoch 9/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 439s 5ms/step - accuracy: 0.9519 - loss: 0.1370
Epoch 10/10
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 461s 5ms/step - accuracy: 0.9521 - loss: 0.1366
94934/94934 ━━━━━━━━━━━━━━━━━━━━ 226s 2ms/step
13524/13524 ━━━━━━━━━━━━━━━━━━━━

In [7]:
np.save('cnn_features_train.npy', cnn_features)
np.save('cnn_features_cnn_test.npy', cnn_features_test)
features = {'train': cnn_features, 'test': cnn_features_test}
np.save('cnn_features.npy', features, allow_pickle=True)
cnn_model.save('cnn_model.h5')

In [12]:
from IPython.display import FileLink
FileLink(r'cnn_features_train.npy')

/kaggle/working/cnn_features_train.npy

In [13]:
from IPython.display import FileLink
FileLink(r'cnn_features_cnn_test.npy')

/kaggle/working/cnn_features_cnn_test.npy

In [14]:
from IPython.display import FileLink
FileLink(r'cnn_features.npy')

/kaggle/working/cnn_features.npy

In [15]:
from IPython.display import FileLink
FileLink(r'cnn_model.h5')

/kaggle/working/cnn_model.h5